In [18]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import json
# Set your OpenAI API key
# import os#
# from dotenv import load_dotenv
# load_dotenv()
import os
from dotenv import load_dotenv

openai_key = os.getenv("OPENAI_API_KEY")
embedding = OpenAIEmbeddings(api_key=openai_key)
 
# Specify your PDF folder path
pdf_folder_path = r"C:\Users\surya_chakka\BD_Assistant\CaseStudies"
 
# Load all PDFs from the folder
# documents = []
# for file in os.listdir(pdf_folder_path):
#     if file.endswith('.pdf'):
#         pdf_path = os.path.join(pdf_folder_path, file)
#         loader = PyMuPDFLoader(pdf_path)
#         documents.extend(loader.load())
# ... existing imports ...
 
# Load first 2 pages from all PDFs in the folder
# ... existing imports ...
 
documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        try:
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyMuPDFLoader(pdf_path)
            # Load all pages
            pages = loader.load()
            print(f"Processing {file}: {len(pages)} pages loaded")
            documents.extend(pages)
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")
            continue
 
# Add a check before proceeding
if not documents:
    raise ValueError("No documents were successfully loaded")
 
# ... rest of the code remains same ...
# ... rest of the code remains same ...
 
# ... rest of the code remains same ...
# Split documents into chunks (optional but recommended)
from langchain.text_splitter import RecursiveCharacterTextSplitter
 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " "]
)
split_documents = text_splitter.split_documents(documents)
 
# Create vector store
db = FAISS.from_documents(split_documents, embedding)
 
# Save the vector store locally (optional)
db.save_local("./Index")
 
# Print some stats
print(f"Loaded {len(documents)} PDF documents")
print(f"Created {len(split_documents)} text chunks")

Processing Case Study (1).pdf: 5 pages loaded
Processing Case Study (10).pdf: 4 pages loaded
Processing Case Study (100).pdf: 6 pages loaded
Processing Case Study (101).pdf: 5 pages loaded
Processing Case Study (102).pdf: 6 pages loaded
Processing Case Study (103).pdf: 5 pages loaded
Processing Case Study (104).pdf: 4 pages loaded
Processing Case Study (105).pdf: 4 pages loaded
Processing Case Study (106).pdf: 6 pages loaded
Processing Case Study (107).pdf: 10 pages loaded
Processing Case Study (108).pdf: 7 pages loaded
Processing Case Study (109).pdf: 7 pages loaded
Processing Case Study (11).pdf: 6 pages loaded
Processing Case Study (110).pdf: 5 pages loaded
Processing Case Study (111).pdf: 6 pages loaded
Processing Case Study (112).pdf: 5 pages loaded
Processing Case Study (113).pdf: 5 pages loaded
Processing Case Study (114).pdf: 4 pages loaded
Processing Case Study (115).pdf: 6 pages loaded
Processing Case Study (116).pdf: 3 pages loaded
Processing Case Study (117).pdf: 12 pages l

In [20]:
try:
    db=FAISS.load_local("./Index",embedding)    
except:
    db=FAISS.load_local("./Index",embedding,allow_dangerous_deserialization=True)

In [21]:
user_prompt="Give me the names of all case studies in json format?"
most_similar_chunks = db.similarity_search_with_score(user_prompt,k=10)

In [22]:
most_similar_chunks[0]

(Document(metadata={'source': 'C:\\Users\\surya_chakka\\BD_Assistant\\CaseStudies\\Case Study (147).pdf', 'file_path': 'C:\\Users\\surya_chakka\\BD_Assistant\\CaseStudies\\Case Study (147).pdf', 'page': 0, 'total_pages': 3, 'format': 'PDF 1.7', 'title': 'PowerPoint Presentation', 'author': 'Moumita Gandhi', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'creationDate': "D:20241114131919+05'30'", 'modDate': "D:20241114131919+05'30'", 'trapped': ''}, page_content='Project Name\nClient\nBrief Description\nKey Analyses\nSpend Cube \nPrivate Equity\nConsolidated indirect spend data from multiple data sources and built real time \ndashboards to provide visibility into indirect spend by spend category, sub-\ncategory, vendor, customer and across time periods.\n1\nSUCCESS STORIES - SUMMARY'),
 0.4920848)

In [23]:
aggregated_content = "\n".join([chunk[0].page_content for chunk in most_similar_chunks])
aggregated_content[:1000]

'Project Name\nClient\nBrief Description\nKey Analyses\nSpend Cube \nPrivate Equity\nConsolidated indirect spend data from multiple data sources and built real time \ndashboards to provide visibility into indirect spend by spend category, sub-\ncategory, vendor, customer and across time periods.\n1\nSUCCESS STORIES - SUMMARY\nProject Name\nBrief Description\nKey analyses\nProvided white-label equity research reports by building \nand analysing bottom-up financial models with all \nrevenue and cost drivers projected based on \ncomprehensive understanding of the sector, the company \nand the macroeconomic factors affecting the trends. \nAdditionally, ensured turnaround times of about 4-5 weeks \nfor each stock report.\n1\nBI REPORTING CASE STUDY SUMMARY\n1\nProject Name\nClient\nBrief Description\nKey Analyses\nBusiness Performance Reporting for Theatrical \nProduction company\nTheatrical Production \ncompany\nBuilt executive level performance dashboards and reports at \nlocation and sho

In [24]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# Define the response schema
response_schemas = [
    ResponseSchema(name="case_studies", description="List of case studies with their details"),
    ResponseSchema(name="total_count", description="Total number of case studies analyzed")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Create prompt template
template = """Analyze the 'Situation', 'Value Addition', 'Impact' sections from the second page of every case study or pdf and retrieve the case studies that are relevant to the text provided and format the response as a JSON.
Extract the case study number and name along with the following details:
1. Company Profile: Get company name and summarize the company's key attributes (e.g., industry, scale, market segment, key technologies).
2. Pain Points: Highlight the challenges or issues the company is facing. Include explicit and inferred points seperately.
3. Relevant Domains: Specify the functional areas or domains these challenges belong to.
4. Relevant Solutions: Summarize the solutions that are provided for the pain points in the following pages.

Respond in JSON format with keys: "Company Profile", "Pain Points", "Relevant Domains", "Relevant Solutions".
The response should only be json format that I can directly convert to json file from string.

Text to analyze: {text}

{format_instructions}

Return only the JSON response without any additional text."""

prompt = ChatPromptTemplate.from_template(template)

# Initialize ChatOpenAI
llm = ChatOpenAI(api_key=openai_key, model="gpt-4o-mini", temperature=0)

# Get similar chunks and combine them
user_question = "We have a apperal and lifestyle brand client facing continuous average revenue per customer reduction and seeking help with financial planning and anlysis along with the identification of reasons behind reduction "
similar_chunks = db.similarity_search_with_score(user_question, k=10)
context = "\n".join([chunk[0].page_content for chunk in similar_chunks])

# Format the prompt
format_instructions = output_parser.get_format_instructions()
messages = prompt.format_messages(text=context, format_instructions=format_instructions)

# Get response
response = llm.invoke(messages)
structured_output = output_parser.parse(response.content)

# Print formatted JSON
import json
print(json.dumps(structured_output, indent=2))

{
  "case_studies": [
    {
      "Company Profile": {
        "Company Name": "Leading Apparel Brand",
        "Key Attributes": {
          "Industry": "Apparel and Lifestyle",
          "Scale": "Large, ~100 stores across the U.S.",
          "Market Segment": "Retail and E-commerce",
          "Key Technologies": "Customer Analytics, POS Data"
        }
      },
      "Pain Points": {
        "Explicit": [
          "Limited insights into customer journey and buying preferences",
          "Need for better customer acquisition and retention strategies"
        ],
        "Inferred": [
          "Potential high value customers not being identified",
          "Sales performance tracking at various customer journey stages is lacking"
        ]
      },
      "Relevant Domains": [
        "Customer Analytics",
        "Marketing Strategy",
        "Sales Performance"
      ],
      "Relevant Solutions": [
        "Leveraged customer level POS data for insights",
        "Identified ke